In [19]:
import torch

from rona.data import RonaData
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [12]:
!ls /Users/ygx/data/utkml/utkML_projects/covid_ct_scans/ct_data

COVID     non-COVID


In [13]:
dataroot = Path("/Users/ygx/data/utkml/utkML_projects/covid_ct_scans/ct_data")

In [14]:
data_transforms = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

In [15]:
data = RonaData(dataroot, data_transforms)

In [16]:
l = DataLoader(data, batch_size=5)

In [17]:
next(iter(l))[1]

tensor([1, 0, 1, 0, 1])

In [21]:
stds = []
means = []
for img, _ in data:
    means.append(torch.mean(img))
    stds.append(torch.std(img))

mean = torch.mean(torch.tensor(means))
std = torch.mean(torch.tensor(stds))

In [22]:
mean

tensor(0.6405)

In [23]:
std

tensor(0.2562)